In [1]:

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score,confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, classification_report, confusion_matrix, roc_curve, auc, f1_score


### Funções Random Forest

In [2]:
def RandomForest(df):
    X = df.drop('diagnostico', axis=1)
    y = df['diagnostico']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
    
    params_grid = {'bootstrap': [True, False],
                    'max_depth': [20, 40, 60, 80],
                    'max_features': ['auto', 'sqrt'],
                    'min_samples_leaf': [1, 2, 4],
                    'min_samples_split': [2, 5, 10],
                    'n_estimators': [100, 200]}
    
    rf = RandomForestClassifier()
    grid_classifier = RandomizedSearchCV(estimator = rf, param_distributions = params_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
    grid_classifier.fit(X_val, y_val)
    
    best_parameters = grid_classifier.best_params_
    
    model = RandomForestClassifier(**best_parameters)
    model.fit(X_train, y_train)
    
    pred = model.predict(X_test)
    
    test_accuracy = accuracy_score(y_test, pred)
    cm = confusion_matrix(y_test, pred)
    ppv = precision_score(y_test, pred)
    sensit = recall_score(y_test, pred)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1])
    
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc_score = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, pred)
    print(f"F1-Score: {f1:.4f}")
    print(f"Acurácia: {test_accuracy:.4f}")
    print(f"PPV (Precision): {ppv:.4f}")
    print(f"Sensibilidade (Recall): {sensit:.4f}")
    print(f"Especificidade: {specificity:.4f}")
    print(f"AUC: {auc_score:.4f}")    
    print(classification_report(y_test,pred))
    print(confusion_matrix(y_test, pred))
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_test, pred), annot=True, fmt='d', cmap='Blues', xticklabels=['Sem Sepse', 'Com Sepse'], yticklabels=['Sem Sepse', 'Com Sepse'])
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.title('Matriz de Confusão')
    plt.show()

In [3]:
def RandomForest_Over(df):
    X = df.drop('diagnostico', axis=1)
    y = df['diagnostico']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
    
    # Aplicar oversampling aleatório apenas nos dados de treino
    ros = RandomOverSampler(random_state=42)
    X_train_res, y_train_res = ros.fit_resample(X_train, y_train)
    
    params_grid = {'bootstrap': [True, False],
                    'max_depth': [20, 40, 60, 80],
                    'max_features': ['auto', 'sqrt'],
                    'min_samples_leaf': [1, 2, 4],
                    'min_samples_split': [2, 5, 10],
                    'n_estimators': [100, 200]}
    
    rf = RandomForestClassifier()
    grid_classifier = RandomizedSearchCV(estimator=rf, param_distributions=params_grid, n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)
    grid_classifier.fit(X_val, y_val)
    
    best_parameters = grid_classifier.best_params_
    
    model = RandomForestClassifier(**best_parameters)
    # Treinar o modelo com os dados balanceados
    model.fit(X_train_res, y_train_res)
    
    pred = model.predict(X_test)
    
    test_accuracy = accuracy_score(y_test, pred)
    cm = confusion_matrix(y_test, pred)
    ppv = precision_score(y_test, pred)
    sensit = recall_score(y_test, pred)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1])
    
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc_score = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, pred)
    print(f"F1-Score: {f1:.4f}")
    print(f"Acurácia: {test_accuracy:.4f}")
    print(f"PPV (Precision): {ppv:.4f}")
    print(f"Sensibilidade (Recall): {sensit:.4f}")
    print(f"Especificidade: {specificity:.4f}")
    print(f"AUC: {auc_score:.4f}")  
        
    print(classification_report(y_test, pred))
    print(confusion_matrix(y_test, pred))
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_test, pred), annot=True, fmt='d', cmap='Blues', xticklabels=['Sem Sepse', 'Com Sepse'], yticklabels=['Sem Sepse', 'Com Sepse'])
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.title('Matriz de Confusão')
    plt.show()
    

### Sem Tratar Missing

In [ ]:
df_final = pd.read_csv('df_final.csv')

In [ ]:
RandomForest(df_final)

In [ ]:
RandomForest_Over(df_final)

### Valores Normais

In [ ]:
df_normal = pd.read_csv('df_normal.csv')

In [ ]:
RandomForest(df_normal)

In [ ]:
RandomForest_Over(df_normal)

### KNN

In [ ]:
df_knn = pd.read_csv('df_knn.csv')

In [ ]:
RandomForest(df_knn)

In [ ]:
RandomForest_Over(df_knn)

### Media

In [ ]:
df_media = pd.read_csv('df_media.csv')

In [ ]:
RandomForest(df_media)

In [ ]:
RandomForest_Over(df_media)

### Mediana

In [ ]:
df_mediana = pd.read_csv('df_mediana.csv')

In [ ]:
RandomForest(df_mediana)

In [ ]:
RandomForest_Over(df_mediana)

### Moda

In [ ]:
df_moda = pd.read_csv('df_moda.csv')

In [ ]:
RandomForest(df_moda)

In [ ]:
RandomForest_Over(df_moda)

### Regressão

In [ ]:
df_regressao = pd.read_csv('df_regressao.csv')

In [ ]:
RandomForest(df_regressao)

In [ ]:
RandomForest_Over(df_regressao)